In [5]:
/* Welcome to the SQL mini project. You will carry out this project partly in
the PHPMyAdmin interface, and partly in Jupyter via a Python connection.

This is Tier 1 of the case study, which means that there'll be more guidance for you about how to 
setup your local SQLite connection in PART 2 of the case study. 

The questions in the case study are exactly the same as with Tier 2. 

PART 1: PHPMyAdmin
You will complete questions 1-9 below in the PHPMyAdmin interface. 
Log in by pasting the following URL into your browser, and
using the following Username and Password:

URL: https://sql.springboard.com/
Username: student
Password: learn_sql@springboard

The data you need is in the "country_club" database. This database
contains 3 tables:
    i) the "Bookings" table,
    ii) the "Facilities" table, and
    iii) the "Members" table.

In this case study, you'll be asked a series of questions. You can
solve them using the platform, but for the final deliverable,
paste the code for each solution into this script, and upload it
to your GitHub.

Before starting with the questions, feel free to take your time,
exploring the data, and getting acquainted with the 3 tables. */


/* QUESTIONS 
/* Q1: Some of the facilities charge a fee to members, but some do not.
Write a SQL query to produce a list of the names of the facilities that do. */

%sql
SELECT name, membercost FROM country_club.Facilities AS f WHERE f.membercost != 0

/*output:
Tennis Court 1 5.0
Tennis Court 2 5.0
Massage Room 1 9.9
Massage Room 2 9.9
Squash Court 3.5
*/

/* Q2: How many facilities do not charge a fee to members? */

%sql
SELECT name, membercost FROM country_club.Facilities AS f WHERE f.membercost = 0

/*output: 
Badminton Court 0.0
Table Tennis 0.0
Snooker Table 0.0
Pool Table 0.0
*/


/* Q3: Write an SQL query to show a list of facilities that charge a fee to members,
where the fee is less than 20% of the facility's monthly maintenance cost.
Return the facid, facility name, member cost, and monthly maintenance of the
facilities in question. */

%sql
SELECT facid, name, membercost, monthlymaintenance 
FROM country_club.Facilities AS f 
WHERE 0.20*monthlymaintenance > membercost AND membercost != 0

/*output:
facid, name, membercost, monthlymaintenance 
0 Tennis Court 1 5.0 200
1 Tennis Court 2 5.0 200
4 Massage Room 1 9.9 3000
5 Massage Room 2 9.9 3000
6 Squash Court   3.5 80
*/

/* Q4: Write an SQL query to retrieve the details of facilities with ID 1 and 5.
Try writing the query without using the OR operator. */

%sql
SELECT * FROM `Facilities` WHERE facid = 1 OR facid = 5

/*output:
facid, monthlymaintenance, name, membercost, guestcost, initialoutlay
1 200  Tennis Court 2 5.0 25.0 8000
5 3000 Massage Room 2 9.9 80.0 4000
*/

/* Q5: Produce a list of facilities, with each labelled as
'cheap' or 'expensive', depending on if their monthly maintenance cost is
more than $100. Return the name and monthly maintenance of the facilities
in question. */

%sql
SELECT name, monthlymaintenance,
CASE WHEN monthlymaintenance > 100 THEN 'expensive'
     WHEN monthlymaintenance < 100 THEN 'cheap'
	 ELSE 'boundary' END AS 'cost'
FROM `Facilities`

/*output:
name, monthlymaintenance, cost
Tennis Court 1 200 expensive
Tennis Court 2 200 expensive
Badminton Court 50 cheap 
Table Tennis 10 cheap
Massage Room 1 3000 expensive
Massage Room 2 3000 expensive
Squash Court 80 cheap 
Snooker Table 15 cheap
Pool Table 15 cheap
*/

/* Q6: You'd like to get the first and last name of the last member(s)
who signed up. Try not to use the LIMIT clause for your solution. */

%sql
SELECT surname, firstname, joindate FROM `Members` ORDER BY joindate DESC

/*output:
Darren Smith
*/

/* Q7: Produce a list of all members who have used a tennis court.
Include in your output the name of the court, and the name of the member
formatted as a single column. Ensure no duplicate data, and order by
the member name. */

%sql
SELECT DISTINCT surname, firstname, Bookings.facid + 1 AS 'Tennis Court'
FROM Members
INNER JOIN Bookings
ON Members.memid = Bookings.memid
WHERE Bookings.facid = 0 OR Bookings.facid = 1
ORDER BY Members.surname

/*
output:
surname, firstname, Tennis Court
Bader Florence 2
Bader Florence 1
Baker Timothy 2
Baker Anne 1
Baker Anne 2
Baker Timothy 1
Boothe Tim 1
Boothe Tim 2
Butters Gerald 1
Butters Gerald 2
Coplin Joan 1
etc
*/

/* Q8: Produce a list of bookings on the day of 2012-09-14 which
will cost the member (or guest) more than $30. Remember that guests have
different costs to members (the listed costs are per half-hour 'slot'), and
the guest user's ID is always 0. Include in your output the name of the
facility, the name of the member formatted as a single column, and the cost.
Order by descending cost, and do not use any subqueries. */

%sql
SELECT Members.surname AS MemberName, Facilities.name AS Facility,
  CASE 
	WHEN Members.memid != 0 THEN Bookings.slots*Facilities.membercost
    WHEN Members.memid = 0 THEN Bookings.slots*Facilities.guestcost
    ELSE  "NULL" END AS Cost
FROM Members
JOIN Bookings 
ON Members.memid = Bookings.memid
JOIN Facilities 
ON Bookings.facid = Facilities.facid
WHERE Bookings.starttime LIKE '2012-09-14%'
AND ((Members.memid != 0 AND Bookings.slots*Facilities.membercost > 30)
       OR 
     (Members.memid = 0 AND Bookings.slots*Facilities.guestcost > 30)) 
ORDER BY Cost DESC

/*output:
MemberName, Facility, Cost
GUEST Tennis Court 1 75.0
GUEST Tennis Court 2 75.0
GUEST Tennis Court 1 75.0
GUEST Squash Court 70.0
Farrell Massage Room 1 39.6
GUEST Squash Court 35.0
GUEST Squash Court 35.0
GUEST Massage Room 2 320.0
GUEST Massage Room 1 160.0
GUEST Massage Room 1 160.0
GUEST Massage Room 1 160.0
GUEST Tennis Court 2 150.0
    etc
*/
    
/* Q9: This time, produce the same result as in Q8, but using a subquery. */

%sql
    SELECT MemberName, Facility, Cost 
FROM
	(SELECT Members.surname AS MemberName, 
            Facilities.name AS Facility,
  			CASE WHEN Members.memid = 0 THEN Bookings.slots*Facilities.guestcost
    		 	 WHEN Members.memid != 0 THEN Bookings.slots*Facilities.membercost
     		ELSE 'Null' END AS Cost
	 FROM Members
	 JOIN Bookings   
     ON Members.memid = Bookings.memid
     JOIN Facilities 
     ON Facilities.facid = Bookings.facid
     WHERE Bookings.starttime LIKE '2012-09-14%'
	) AS subquery
WHERE Cost > 30 
ORDER BY Cost DESC

/* PART 2: SQLite */
/* We now want you to jump over to a local instance of the database on your machine. 
Copy and paste the LocalSQLConnection.py script into an empty Jupyter notebook, and run it. 
Make sure that the SQLFiles folder containing thes files is in your working directory, and
that you haven't changed the name of the .db file from 'sqlite\db\pythonsqlite'.
You should see the output from the initial query 'SELECT * FROM FACILITIES'.
Complete the remaining tasks in the Jupyter interface. If you struggle, feel free to go back
to the PHPMyAdmin interface as and when you need to. 
You'll need to paste your query into value of the 'query1' variable and run the code block again to get an output.
*/
 
/*QUESTIONS:*/

/* Q10: Produce a list of facilities with a total revenue less than 1000.
The output of facility name and total revenue, sorted by revenue. Remember
that there is a different cost for guests and members! */

%sql
SELECT name, revenue 
FROM 
  (SELECT Facilities.name,
    SUM(CASE WHEN Bookings.memid = 0 THEN Bookings.slots*Facilities.guestcost
             WHEN Bookings.memid != 0 THEN Bookings.slots*Facilities.membercost
        	 ELSE "Null" END) AS revenue 
    FROM Bookings
    JOIN Facilities 
    ON Bookings.facid = Facilities.facid
    GROUP BY Facilities.name) AS subquery 
WHERE revenue < 1000
ORDER BY revenue;

/*
output:
name, revenue
Table Tennis 180
Snooker Table 240
Pool Table 270
*/

/* Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order */

%sql
SELECT CONCAT(m.firstname,' ',m.surname) as Recommended_By,
	   CONCAT(rcmd.firstname,' ',rcmd.surname) as Member
FROM Members m
INNER JOIN Members rcmd 
ON rcmd.recommendedby = m.memid
WHERE m.memid != 0
ORDER BY m.surname,m.firstname,rcmd.surname,rcmd.surname

/* 
output:
Recommended_By, Member
Florence Bader Ramnaresh Sarwin
Timothy Baker Joan Coplin
Gerald Butters Matthew Genting
Jemima Farrell Timothy Baker
Jemima Farrell David Pinker
Matthew Genting Henrietta Rumney
David Jones Douglas Jones
Janice Joplette Nancy Dare
Janice Joplette David Jones
Millicent Purview John Hunt
Tim Rownam Tim Boothe
Darren Smith Gerald Butters
Darren Smith Janice Joplette
Darren Smith Anna Mackenzie
Darren Smith Charles Owen
Darren Smith Jack Smith
Tracy Smith Erica Crumpet
Tracy Smith Millicent Purview
Tracy Smith Henry Worthington-Smyth
Ponder Stibbons Florence Bader
Ponder Stibbons Anne Baker
Burton Tracy Ponder Stibbons
    etc
*/

/* Q12: Find the facilities with their usage by member, but not guests.
COUNT is the number of records, SUM is what is what is in the record*/

%sql
SELECT f.name,
	   CONCAT(m.firstname,' ',m.surname) as member,
       COUNT(f.name) as bookings
FROM Members m
INNER JOIN Bookings bk 
ON bk.memid = m.memid
INNER JOIN Facilities f 
ON f.facid = bk.facid
WHERE m.memid != 0
GROUP BY f.name, concat(m.firstname,' ',m.surname)
ORDER BY f.name, m.surname, m.firstname 

/*
output:
name, member, bookings
Badminton Court Florence Bader 9
Badminton Court Anne Baker 10
Badminton Court Timothy Baker 7
Badminton Court Tim Boothe 12
Badminton Court Gerald Butters 20
Badminton Court Erica Crumpet 2
Badminton Court Nancy Dare 10
Badminton Court Jemima Farrell 7
Badminton Court John Hunt 2
Badminton Court David Jones 8
Badminton Court Douglas Jones 2
Badminton Court Anna Mackenzie 30
    etc
*/

/* Q13: Find the facilities usage by month, but not guests */

%sql
SELECT f.name,concat(m.firstname,' ',m.surname) AS member,
COUNT(f.name) AS bookings,
SUM(CASE WHEN MONTH(starttime) = 1 then 1 else 0 end) AS Jan,
SUM(CASE WHEN MONTH(starttime) = 2 then 1 else 0 end) AS Feb,
SUM(CASE WHEN MONTH(starttime) = 3 then 1 else 0 end) AS Mar,
SUM(CASE WHEN MONTH(starttime) = 4 then 1 else 0 end) AS Apr,
SUM(CASE WHEN MONTH(starttime) = 5 then 1 else 0 end) AS May,
SUM(CASE WHEN MONTH(starttime) = 6 then 1 else 0 end) AS Jun,
SUM(CASE WHEN MONTH(starttime) = 7 then 1 else 0 end) AS Jul,
SUM(CASE WHEN MONTH(starttime) = 8 then 1 else 0 end) AS Aug,
SUM(CASE WHEN MONTH(starttime) = 9 then 1 else 0 end) AS Sep,
SUM(CASE WHEN MONTH(starttime) = 10 then 1 else 0 end) AS Oct,
SUM(CASE WHEN MONTH(starttime) = 11 then 1 else 0 end) AS Nov,
SUM(CASE WHEN MONTH(starttime) = 12 then 1 else 0 end) AS Dec_
FROM Members m
INNER JOIN Bookings bk 
ON bk.memid = m.memid
INNER JOIN Facilities f 
ON f.facid = bk.facid
WHERE m.memid != 0 AND year(starttime) = 2012
GROUP BY f.name, CONCAT(m.firstname,' ',m.surname)
ORDER BY f.name,m.surname,m.firstname

/*
output:
name, member, bookings, Jan, Feb, Mar, Apr, May, Jun, Jul, Aug, Sep, Oct, Nov, Dec_
Badminton Court Florence Bader 9 0 0 0 0 0 0 0 5 4 0 0 0
Badminton Court Anne Baker 10 0 0 0 0 0 0 0 1 9 0 0 0
Badminton Court Timothy Baker 7 0 0 0 0 0 0 0 3 4 0 0 0
Badminton Court Tim Boothe 12 0 0 0 0 0 0 1 9 2 0 0 0
Badminton Court Gerald Butters 2 0 0 0 0 0 0 0 4 8 8 0 0 0
Badminton Court Erica Crumpet 2 0 0 0 0 0 0 0 0 2 0 0 0
    etc
*/

SyntaxError: invalid syntax (<ipython-input-5-4f72de48e10a>, line 1)